In [ ]:
# AdamModel 클래스 역할 
# 목적 : MlpModel 클래스를 상속받아 Adam 연산 과정을 지원합니다. 
class AdamModel(MlpModel):
    print("AdamModel 호출")
    def __init__(self, model_name, dataset, hconfigs):
        
        # Adam 의 사용 여부 기본값은 False 처리 합니다. 
        # Adam 은 연산과정에서 다소 높은 처리량을 요구합니다. 
        self.use_adam = False 
        super(AdamModel, self).__init__(model_name, dataset, hconfigs)

In [ ]:
# AdamModel.backprop_layer 의 역할 
# 목적 : 
# MlpModel.backprop_layer 의 과정과 매우 흡사하나,
# Adam 을 적용한 경우의 파라미터 업데이트 과정입니다. 
# G_y : G_output
# hconfig : 은닉계층 정보
# pm : 파라미터
# aux : 역전파에 필요한 위한 보조지표
def adam_backprop_layer(self, G_y, hconfig, pm, aux):
    print("AdamModel.backprop_layer 호출")
    
    # MlpModel.backprop_layer 의 과정과 같습니다. 
    x, y = aux
    
    # [마지막 은닉계층과 이전 은닉계층의 ∂L/∂Y (delta_{k+1}) 준비 - 2/2단계]
    # 전체 수식 : (𝛅_k * w_k) * 𝜑(h)
    # 𝜑(h) : 은닉계층에서 비선형 활성화함수 relu가 사용되므로, 이에 따른 미분과정(phi)
    # 2단계에서 구현하는 수식 : (𝛅_k * w_k) * 𝜑(h)
    if hconfig is not None : 
        G_y = G_y * relu_derv(y)
    
    # [출력계층과 마지막 은닉계층 사이의 파라미터 갱신 준비]
    # 가중치 갱신에 필요한 수식 : ∂L/∂W (X^t * G)
    # 가장 마지막 은닉 계층(x.transpose())의 행렬전환 
    g_y_weight = x.transpose()
    g_y_input = pm['w'].transpose()
    
    # 가중치 갱신 준비과정 : ∂L/∂W 
    # X^t * G
    G_weight = np.matmul(g_y_weight, G_y)
    
    # 편향 갱신에 필요한 수식 : ∂L/∂B 
    # \bold{G}
    G_bias = np.sum(G_y, axis=0)
    
    # [마지막 은닉계층과 이전 은닉계층의 ∂L/∂Y (delta_k+1) 준비 - 1/2 단계]
    # 업데이트가 되지 않은 가중치(g_y_input)가 필요 
    G_input = np.matmul(G_y, g_y_input)
    
    # 파라미터를 업데이트 하는 과정은
    # AdamModel 클래스 내의 update_param() 에서 진행합니다. 
    self.update_param(pm = pm, key = 'w', delta = G_weight)
    self.update_param(pm = pm, key = 'b', delta = G_bias)

    return G_input

AdamModel.backprop_layer = adam_backprop_layer

In [ ]:
# AdamModel.update_param 의 역할 
# 목적 : 
# use_adam 플래그 변수 값에 맞춰 self.eval_adam_delta() 동작
# 매개변수 업데이트

def adam_update_param(self, pm, key, delta):
    print("AdamModel.update_param 호출")
    
    if self.use_adam:
        # 본 메서드의 매개변수는 self.eval_adam_delta() 에 그대로 할당합니다. 
        delta = self.eval_adam_delta(pm, key, delta)
    
    # Adam 연산 수식 일부를 반환받은 후 
    # delta 값을 기반으로 파라미터 업데이트를 진행합니다. 
    pm[key] -= self.learning_rate * delta
        
AdamModel.update_param = adam_update_param

In [ ]:
# AdamModel.eval_adam_delta 의 역할 
# 목적 : 
# 아담 알고리즘의 수식을 파라미터에 맞게 적용 합니다. 
# w_{(ij)}^{(t+1)} = w^{(t)}_{ij} + \frac{\eta}{\sqrt{\hat{g}^{(t)}_{ij}+\epsilon}}\hat{v}_{ij}^{(t)}
def adam_eval_adam_delta(self, pm, key, delta):
    print("AdamModel.eval_adam_delta 호출")
    beta_1 = 0.9
    beta_2 = 0.999 
    epsilon = 1.0e-8 

    # vkey = 'vw' or 'vb' 
    # 파라미터 업데이트에 대한 v 키 값 생성 
    
    # gkey = 'gw' or 'gb'
    # 파라미터 업데이트에 대한 g 키 값 생성
    
    # step = 'nw' or 'nb'
    # 파라미터 업데이트에 대한 n 키 값 생성 (업데이트 횟수)
    vkey, gkey, step = 'v' + key, 'g' + key, 'n' + key
    
    # 각 키와 값에 대한 초기화를 위해 
    # pm 변수내에 vkey 가 없는지 확인 후 초기화 변수 생성 
    if vkey not in pm:
        
        # Adam 내의 v,g 값은 가중치와 편향 shape 에 맞게 0 으로 초기화 
        pm[vkey] = np.zeros(pm[key].shape) 
        pm[gkey] = np.zeros(pm[key].shape)
        pm[step] = 0  # 이 값은 본 메서드의 동작 횟수 

    # v_it^(t) 
    # v^{(t)}_{ij} = \beta_1v_{ij}^{(t-1)} + (1-\beta_1)(\frac{\partial L}{\partial w_{ij}^{(t)}})
    v = pm[vkey] = beta_1 * pm[vkey] + (1 - beta_1) * delta
    
    # g_it^(t)
    # g^{(t)}_{ij} = \beta_2 g_{ij}^{(t-1)} + (1-\beta_2)(\frac{\partial L}{\partial w_{ij}^{(t)}})^2
    g = pm[gkey] = beta_2 * pm[gkey] + (1 - beta_2) * (delta * delta)
    
    pm[step] += 1

    # \hat{v}
    # \hat{v}_{ij}^{(t)} = \frac{v_{ij}^{(t)}}{1-\beta^{t}_1}
    v = v / (1 - np.power(beta_1, pm[step]))

    # \hat{g}
    # \hat{g}_{ij}^{(t)} = \frac{g_{ij}^{(t)}}{1-\beta^{t}_2}
    g = g / (1 - np.power(beta_2, pm[step]))

    # 파라미터 업데이트 수식 일부
    return v / (np.sqrt(g) + epsilon)


AdamModel.eval_adam_delta = adam_eval_adam_delta